# Model Evaluation
Evaluating will be made up of the following two testing metrics for the 3 models.
1. Silhouette Score
2. Genre Evaluation

Given that we were unable to train the hierarchical clustering model on the entire dataset (due to its relative inefficiency) we wll stick to evaluating the k-means and fuzzy c-means models.

In [23]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.spatial.distance import correlation
from tqdm.notebook import tqdm
import pickle
import os

# Load in Data

## Load song feature data 

In [27]:
data = pd.read_csv('data/dataset.csv', index_col=0)
data.drop_duplicates(subset='track_id', inplace=True)
features = ['track_id', 'track_name', 'artists', 'popularity', 'track_genre', 'danceability', 'loudness', 'acousticness', 'valence', 'tempo']
X = data[features].copy()
X.drop_duplicates(subset=['track_name', 'artists'], keep='first', inplace=True)
X.reset_index(drop=True, inplace=True)  # fix indices after removing duplicates

# Standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X[['popularity', 'danceability', 'loudness', 'acousticness', 'valence', 'tempo']])
cos_scaled = X_scaled / np.linalg.norm(X_scaled, axis=1)[:, np.newaxis]

In [3]:
data.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


# Load in Models

In [20]:
with open('models/eucd.pkl', 'rb') as file:
    euc_model = pickle.load(file)
with open('models/cos.pkl', 'rb') as file:
    cos_model = pickle.load(file)
with open('models/fuzzy.pkl', 'rb') as file:
    (cntr, u) = pickle.load(file)

In [70]:
results = X[['track_id', 'artists', 'track_name']].copy()
results['euc_cluster'] = euc_model.predict(X_scaled).astype(str)
results['cos_cluster'] = cos_model.predict(cos_scaled).astype(str)

## Create results matrix for the cosine and euclidean models

# 1. Compute Silhouette Scores 

# 2. Compare with User-made Playlists

## Load user playlist data

In [ ]:
# Load first 1 million data points, excluding the user id
playlists = pd.read_csv('data/spotify_dataset.csv', on_bad_lines='skip', usecols=[' "artistname"', ' "trackname"', ' "playlistname"'])

# Rename columns
playlists.rename(columns={' "artistname"': 'artists', ' "trackname"': 'track_name', ' "playlistname"': 'playlist'}, inplace=True)

# Remove duplicates from playlists
playlists.drop_duplicates(subset=['track_id', 'playlist'])

# Remove all parantheticals in song names to treat more songs as the same
playlists['track_name'].str.replace(r"\(.*\)","", regex=True)

## Find all the playlists containing songs in the original playlist

In [98]:
mixed_data = pd.merge(X, playlists, how='left', on=['artists', 'track_name'])
mixed_data.drop_duplicates(subset=['track_id', 'playlist'], inplace=True)

## Evaluate models based on playlist prediction

In [104]:
evals = 10  # Number of songs to consider
k = 5  # Number of songs to predict based on the given playlist
p_length = 5  # Number of songs to pick from the playlist

valid_lists = mixed_data.groupby(by='playlist').filter(lambda g: len(g) >= p_length + k)
valid_playlists = valid_lists['playlist'].unique()

In [111]:
percentages = []
np.random.seed(42)
for _ in tqdm(range(evals)):
    # Sample random playlist from the possible playlists
    play = np.random.choice(valid_playlists)

    # Sample a random playlist
    total_playlist = valid_lists[valid_lists.playlist == play]

    # Sample `p_length` songs from that playlist
    playlist = valid_lists[valid_lists.playlist == play].sample(p_length).track_id

    # Look up those songs in the catelog based on their id and find their index
    playlist = results[results.track_id.isin(playlist)].index

    if len(playlist) == 0:
        display(total_playlist)

    # Find the majority cluster of the songs in the playlist
    clusters = results.iloc[playlist]['euc_cluster']
    majority = np.argmax(np.bincount(clusters))

    # For each song in majority cluster, compute the correlation distance to each song in the playlist
    song_score = []
    indices = results[results['euc_cluster'] == str(majority)].index
    N = len(playlist)
    for song in indices:
        score = float('inf')
        for liked_song in playlist:
            score = min(score, correlation(X_scaled[song], X_scaled[liked_song])) # min(score, np.mean((X_scaled[song] - X_scaled[liked_song])**2))
        song_score.append(score)

    # Find the `k` most similar songs based on the correlation distance
    most_similar = np.argsort(song_score)[p_length:k+p_length]
    songs = results.iloc[indices[most_similar]]

    # Compute the percentage of the songs in the user-made playlists
    inn = total_playlist[total_playlist.track_id.isin(songs)].count()
    display(X.iloc[indices[most_similar]])
    display(total_playlist)
    percentages.append(inn / (len(total_playlist) - p_length))
        

  0%|          | 0/10 [00:00<?, ?it/s]

,track_id,track_name,artists,popularity,track_genre,danceability,loudness,acousticness,valence,tempo
73903,2DNyZP4Py6f4zMASLBnIu6,La Playa,La Oreja de Van Gogh,74,spanish,0.589,-5.707,0.175000,0.311,88.535
73912,46Yp4JsZIh8ceg5WBqW1ZB,Buscando en la basura,La Fuga,57,spanish,0.557,-3.809,0.005310,0.647,95.025
73926,05HpBFLrHBjWpgV0DSPUAv,Tiroteo,Marc Seguí;Pol Granch,69,spanish,0.747,-6.633,0.054800,0.564,110.402
73928,55DfugJIh1351fCiwO2we2,La Tormenta De Arena,Dorian,64,spanish,0.603,-2.952,0.000048,0.548,127.983
73934,6hpxFeXdcn1cvjtxXc7NSD,AN1MAL,Lola Indigo,60,spanish,0.767,-5.406,0.089300,0.585,124.972


,track_id,track_name,artists,popularity,track_genre,danceability,loudness,acousticness,valence,tempo,playlist
41291,2aibwv5hGXSgw7Yru8IYTO,Snow (Hey Oh),Red Hot Chili Peppers,80,alt-rock,0.427,-3.674,0.116000,0.599,104.655,Rrahn-Rrahn!
80681,3FtYbEfBqAlGO46NUDQSAt,Electric Feel,MGMT,77,alt-rock,0.763,-3.713,0.070900,0.561,103.040,Rrahn-Rrahn!
118942,0SShlR5RlMpTzifqbPnVhl,In For The Kill,La Roux,0,alternative,0.627,-0.223,0.001370,0.894,150.063,Rrahn-Rrahn!
183303,2JpP6bBwQGtwAf3CkE46NY,Sometimes I Feel So Deserted,The Chemical Brothers,36,breakbeat,0.671,-6.204,0.007810,0.450,119.984,Rrahn-Rrahn!
250083,0ct6r3EGTcMLPtrXHDvVjc,The Nights,Avicii,86,dance,0.527,-5.298,0.016600,0.654,125.983,Rrahn-Rrahn!
265290,0g2zW7vCyWkUJg5aQjOwi7,The Days,Avicii,0,dance,0.590,-3.278,0.040000,0.526,127.050,Rrahn-Rrahn!
275136,18XA6zBzc8shI011scLk5k,Let Me Love You (Until You Learn To Love Yours...,Ne-Yo,0,dance,0.669,-5.200,0.213000,0.269,124.925,Rrahn-Rrahn!
311050,0kLrrh7Z1gySNTunBH790J,Sometimes,Miami Horror,59,disco,0.580,-4.994,0.017600,0.430,124.992,Rrahn-Rrahn!
325703,4lhqb6JvbHId48OUJGwymk,Hey Brother,Avicii,77,edm,0.545,-4.867,0.030900,0.458,125.014,Rrahn-Rrahn!
338401,5mc6EyF1OIEOhAkD0Gg9Lc,Closer,Nine Inch Nails,66,electronic,0.546,-8.639,0.000273,0.586,89.991,Rrahn-Rrahn!


,track_id,track_name,artists,popularity,track_genre,danceability,loudness,acousticness,valence,tempo
56064,3RE7RWD5J1zndnSL4WzZ2b,Janeiro,Georgia Castro,42,mpb,0.607,-9.501,0.518,0.4050,136.019
56061,2nGWNPH4uIQJnieJZRktab,Encontro Das Águas - Ao Vivo No Rio De Janeiro...,Jorge Vercillo,41,mpb,0.368,-12.188,0.730,0.0955,81.630
56060,4AVkBP6hLC9dORa6gWoC3R,Digitais - Ao Vivo,Isabella Taviani;Moska,40,mpb,0.335,-8.707,0.569,0.2970,89.274
56058,5Dje6vp12fGgkoXQpKZfq8,Carta de Amor (Ao Vivo),Maria Bethânia,41,mpb,0.616,-6.170,0.562,0.4660,100.848
56057,04SZq5X98v2Gsf1NUX7OFH,Sentimental - Ao Vivo,Los Hermanos,41,mpb,0.305,-12.467,0.503,0.1480,124.484


,track_id,track_name,artists,popularity,track_genre,danceability,loudness,acousticness,valence,tempo,playlist
125935,1vWtTis3VRkkUp8qST4jY5,Dance For Me Wallis,Abel Korzeniowski,58,ambient,0.397,-14.694,0.88900,0.1600,86.043,Imagine anything else
130090,3gxwWtR114M0fzmSPNWdfI,Charms,Abel Korzeniowski,51,ambient,0.241,-13.571,0.92900,0.0650,89.802,Imagine anything else
378386,7x46YkKAwp3yZpaa72n5Mu,Stubborn Love,The Lumineers,68,folk,0.596,-11.759,0.60400,0.2620,115.022,Imagine anything else
526747,392OK8Qdmos0wNDrduiwy0,In Your Own Sweet Way,Wes Montgomery,47,guitar,0.566,-21.919,0.99200,0.0785,122.274,Imagine anything else
593557,5oRHGXBUNsioP4HyhJl3hI,Saku,Susumu Yokota,18,idm,0.395,-17.997,0.83300,0.1150,118.257,Imagine anything else
593561,7GSi6HudnZ05T3qByRfYOZ,Kirakiraboshi,Susumu Yokota,18,idm,0.402,-18.471,0.92800,0.1210,104.169,Imagine anything else
593648,1dNX6lUalqsdphlkFDMNGm,Azukiiro No Kaori,Susumu Yokota,18,idm,0.323,-18.776,0.97700,0.0680,69.822,Imagine anything else
594883,4km48etH1GAlDCVa6Lli37,Genshi,Susumu Yokota,12,idm,0.441,-17.428,0.26100,0.2190,120.003,Imagine anything else
628906,3zIuHdD8dkh1vPxQu334T7,Blue Train,John Coltrane,59,jazz,0.503,-15.425,0.64300,0.5690,136.098,Imagine anything else
920280,5IpTvvTopPoAhJLbtQ798i,Eden,Hooverphonic,49,trip-hop,0.246,-9.576,0.02050,0.1690,82.070,Imagine anything else


...

,track_id,track_name,artists,popularity,track_genre,danceability,loudness,acousticness,valence,tempo,playlist
24704,4CeeEOM32jQcH3eN9Q2dGj,Smells Like Teen Spirit,Nirvana,83,alt-rock,0.502,-4.556,0.000025,0.720,116.761,THE OPTIMISTIC
85833,20I8RduZC2PWMWTDCZuuAN,Take Me Out,Franz Ferdinand,79,alt-rock,0.277,-8.821,0.000409,0.527,104.561,THE OPTIMISTIC
254295,0nrRP2bk19rLc0orkWPQk2,Wake Me Up,Avicii,85,dance,0.532,-5.697,0.003800,0.643,124.080,THE OPTIMISTIC
289263,0rYHxEGSSNtH6UYGsZ9FqW,Fresh,Kool & The Gang,5,disco,0.845,-15.242,0.009290,0.930,116.872,THE OPTIMISTIC
289582,1MrcbZai6wumuSm6JJ6BUA,I Will Survive,Gloria Gaynor,0,disco,0.770,-9.571,0.022700,0.800,116.917,THE OPTIMISTIC
292101,4y1LsJpmMti1PfRQV9AWWe,Girls Just Want to Have Fun,Cyndi Lauper,79,disco,0.710,-4.897,0.220000,0.725,120.372,THE OPTIMISTIC
303637,6pPWRBubXOBAHnjl5ZIujB,The Best,Tina Turner,69,disco,0.662,-9.540,0.108000,0.744,103.867,THE OPTIMISTIC
739064,502YAAGo4tiKD8xIqa3M5N,Friday I'm In Love,The Cure,1,punk-rock,0.552,-4.249,0.001060,0.458,135.979,THE OPTIMISTIC
742658,1QFh8OH1e78dGd3VyJZCAC,Boys Don't Cry,The Cure,77,punk-rock,0.462,-5.766,0.005170,0.671,168.766,THE OPTIMISTIC
756211,6nTiIhLmQ3FWhvrGafw2zj,American Idiot,Green Day,79,punk,0.380,-2.042,0.000026,0.769,186.113,THE OPTIMISTIC


,track_id,track_name,artists,popularity,track_genre,danceability,loudness,acousticness,valence,tempo
54608,2j02RtZz6v53FRw4oWliZf,BURN IT,FEVER 333,56,metalcore,0.591,-3.074,0.003430,0.426,130.017
54610,3p3t4C8mvtZrFBRs6OE7eY,</c0de>,Motionless In White,56,metalcore,0.488,-3.270,0.000023,0.569,108.035
54614,42lC6bnDdAjRqZtsFe1tmg,The Arms of Sorrow,Killswitch Engage,53,metalcore,0.499,-2.729,0.000065,0.160,123.981
54615,4FtWn9oF9YneHguKyz6YYJ,The Negative,I Prevail,58,metalcore,0.549,-4.177,0.000801,0.481,137.912
54582,0v1QET6Q1YD7HQK4FBT8on,Burial Plot,Dayseeker,60,metalcore,0.428,-4.325,0.039000,0.222,92.558


,track_id,track_name,artists,popularity,track_genre,danceability,loudness,acousticness,valence,tempo,playlist
27316,2AEdT2ZKd14kzOouzdNOUj,Californication,Red Hot Chili Peppers,2,alt-rock,0.592,-2.788,0.002100,0.327,96.483,"Unknown, 1."
29309,6ehaWPjsNSf5HWOXTEFSUi,Somebody Told Me,The Killers,0,alt-rock,0.508,-3.230,0.000082,0.746,138.011,"Unknown, 1."
41075,2aibwv5hGXSgw7Yru8IYTO,Snow (Hey Oh),Red Hot Chili Peppers,80,alt-rock,0.427,-3.674,0.116000,0.599,104.655,"Unknown, 1."
55686,3ZOEytgrvLwQaqXreDs2Jx,Can't Stop,Red Hot Chili Peppers,82,alt-rock,0.618,-3.442,0.017900,0.875,91.455,"Unknown, 1."
65148,64BbK9SFKH2jk86U3dGj2P,Otherside,Red Hot Chili Peppers,80,alt-rock,0.458,-3.265,0.003160,0.513,123.229,"Unknown, 1."
71768,1jJci4qxiYcOHhQR247rEU,Kids,MGMT,78,alt-rock,0.451,-3.871,0.000760,0.172,122.961,"Unknown, 1."
79967,3FtYbEfBqAlGO46NUDQSAt,Electric Feel,MGMT,77,alt-rock,0.763,-3.713,0.070900,0.561,103.040,"Unknown, 1."
91954,1sTsuZTdANkiFd7T34H3nb,Human,The Killers,75,alt-rock,0.557,-8.477,0.001320,0.582,135.470,"Unknown, 1."
103020,4iG2gAwKXsOcijVaVXzRPW,Time to Pretend,MGMT,72,alt-rock,0.438,-3.249,0.000087,0.421,100.990,"Unknown, 1."
412713,5XeFesFbtLpXzIVDNQP22n,I Wanna Be Yours,Arctic Monkeys,92,garage,0.464,-9.345,0.136000,0.479,67.528,"Unknown, 1."


,track_id,track_name,artists,popularity,track_genre,danceability,loudness,acousticness,valence,tempo
54608,2j02RtZz6v53FRw4oWliZf,BURN IT,FEVER 333,56,metalcore,0.591,-3.074,0.003430,0.426,130.017
54610,3p3t4C8mvtZrFBRs6OE7eY,</c0de>,Motionless In White,56,metalcore,0.488,-3.270,0.000023,0.569,108.035
54614,42lC6bnDdAjRqZtsFe1tmg,The Arms of Sorrow,Killswitch Engage,53,metalcore,0.499,-2.729,0.000065,0.160,123.981
54615,4FtWn9oF9YneHguKyz6YYJ,The Negative,I Prevail,58,metalcore,0.549,-4.177,0.000801,0.481,137.912
54582,0v1QET6Q1YD7HQK4FBT8on,Burial Plot,Dayseeker,60,metalcore,0.428,-4.325,0.039000,0.222,92.558


,track_id,track_name,artists,popularity,track_genre,danceability,loudness,acousticness,valence,tempo,playlist
289031,4PTG3Z6ehGkBFwjybzWkR8,Never Gonna Give You Up,Rick Astley,45,disco,0.721,-11.823,0.11500,0.9140,113.309,Private Sessions (guilty pleasures)
291464,3olXS7r5RD0pxmBEnI7GGV,Dancing On The Ceiling,Lionel Richie,0,disco,0.688,-6.456,0.12100,0.7090,133.214,Private Sessions (guilty pleasures)
295587,2JoZzpdeP2G6Csfdq5aLXP,How Deep Is Your Love,Bee Gees,74,disco,0.633,-9.366,0.10500,0.6740,104.938,Private Sessions (guilty pleasures)
296623,17CPezzLWzvGfpZW6X8XT0,"Say You, Say Me",Lionel Richie,69,disco,0.636,-11.113,0.35500,0.2570,128.068,Private Sessions (guilty pleasures)
296879,6HMvJcdw6qLsyV1b5x29sa,Hello,Lionel Richie,68,disco,0.424,-15.053,0.64600,0.0606,61.648,Private Sessions (guilty pleasures)
298197,7kAzaxFL9SNlDnCOFENybw,True,Spandau Ballet,0,disco,0.744,-9.361,0.50400,0.3450,97.415,Private Sessions (guilty pleasures)
303002,1pTGc8pwyo6xtgXBKCBcFn,Woman in Love,Barbra Streisand,68,disco,0.469,-16.311,0.24800,0.3310,169.735,Private Sessions (guilty pleasures)
303981,6Hu6dzwlvoyg3zBUC8k4BK,Sailing,Christopher Cross,71,disco,0.469,-13.203,0.36400,0.1780,149.994,Private Sessions (guilty pleasures)
309045,5yGTQzYbEdY6B9RFZJypgt,Rhythm Of The Night,DeBarge,67,disco,0.707,-12.328,0.07690,0.9550,115.142,Private Sessions (guilty pleasures)
310202,4PaDoGTeE3PPyHPfZ3nqaD,I Can't Wait,Nu Shooz,0,disco,0.784,-4.630,0.16600,0.9400,103.972,Private Sessions (guilty pleasures)


In [112]:
np.mean(percentages)

0.0